In [1]:
from bs4 import BeautifulSoup

In [2]:
import requests

In [3]:
import numpy as np

In [4]:
import csv

In [5]:
import pandas as pd

In [6]:
html_text = requests.get('https://www.dsebd.org/latest_share_price_scroll_l.php').text

In [7]:
soup = BeautifulSoup(html_text, 'lxml')

In [8]:
stocks_table = soup.find('div', class_ = 'table-responsive inner-scroll' )

In [9]:
stocks = stocks_table.find_all('tr')

In [10]:
stocks_data_list = []
loop_counter = 0
for stock in stocks:    
    
    stock_data = stock.find_all('td')
    
    stock_data_list = []
    nested_loop_counter = 0
    
    for data in stock_data:
            
        if 1 < nested_loop_counter < 8 :
            stock_data_list += [float(data.text.replace('--' , '0').replace(',',''))]
            
        elif nested_loop_counter == 1:
            stock_name = data.find('a')
            stock_data_list += [stock_name.text.strip()]
            
        elif nested_loop_counter == 8:
            stock_data_list += [int(data.text.replace(',','').replace('--', '0'))]
            
        elif nested_loop_counter == 9:
            stock_data_list += [float(data.text.replace(',','').replace('--','0'))]
                
        elif nested_loop_counter == 10:
            volume = int(data.text.replace(',',''))
            stock_data_list += [volume]
                
        nested_loop_counter += 1
        
    stocks_data_list.append(stock_data_list)

        
del stocks_data_list[0]

In [11]:
stocks_data_list[0]

['1JANATAMF', 6.5, 6.6, 6.5, 0.0, 6.5, 0.0, 16, 0.491, 75570]

In [12]:
stocks_data_list[0][1]

6.5

In [13]:
market_update_dict = dict()
for stock in stocks_data_list:
    if stock[1] == 0:
        continue
    
    elif stock[1] > 0:
        market_update_dict[stock[0]] = stock

In [14]:
listings = open('listings.csv', 'r')
market_update_for_wrangled = open('market_update_wrangled.csv', 'w')

with listings, market_update_for_wrangled:
    read_listings = csv.reader(listings)
    write_market_update = csv.writer(market_update_for_wrangled)
    
    for record in read_listings:
        try:
            write_market_update.writerow(market_update_dict[record[0]])
        except KeyError:
            continue

In [15]:
stocks_data_headings = stocks[0].find_all('th')

In [16]:
del stocks_data_headings[0]

stocks_headings_list = []

for heading in stocks_data_headings:
    stocks_headings_list += [heading.text]

In [17]:
stocks_headings_list

['TRADING CODE',
 'LTP*',
 'HIGH',
 'LOW',
 'CLOSEP*',
 'YCP*',
 'CHANGE',
 'TRADE',
 'VALUE (mn)',
 'VOLUME']

In [18]:
len(stocks_data_list)

384

In [19]:
df_mp_update = pd.read_csv('market_update_wrangled.csv', names = stocks_headings_list)

In [20]:
df_fin_disclosures = pd.read_csv('disclosures_wrangled.csv', names = ['TRADING CODE', 'SECTOR', 'CATEGORY', 'DIVIDEND', 'EPS_Q', 'EPS_Y', 'NAVPS', 'FIN_YEAR'])

In [21]:
df_merged_mp_disclosures = pd.merge(df_mp_update, df_fin_disclosures, on = 'TRADING CODE')

In [22]:
df_merged_mp_disclosures

,TRADING CODE,LTP*,HIGH,LOW,CLOSEP*,YCP*,CHANGE,TRADE,VALUE (mn),VOLUME,SECTOR,CATEGORY,DIVIDEND,EPS_Q,EPS_Y,NAVPS,FIN_YEAR
0,1JANATAMF,6.5,6.6,6.5,0.0,6.5,0.0,16,0.491,75570,Mutual Funds,A,1.30,0.78,2.54,11.94,2021
1,1STPRIMFMF,18.2,18.4,18.0,0.0,18.2,0.0,75,1.534,84540,Mutual Funds,A,0.80,1.40,0.38,11.62,2020
2,AAMRANET,47.2,47.9,47.2,0.0,48.1,-0.9,598,45.722,965156,IT Sector,A,2.65,1.88,2.14,37.09,2021
3,AAMRATECH,38.8,39.7,38.7,0.0,39.4,-0.6,491,54.240,1397757,IT Sector,A,2.21,2.32,1.45,23.93,2021
4,ABB1STMF,5.8,5.8,5.8,0.0,5.7,0.1,10,0.273,47000,Mutual Funds,A,0.80,0.92,2.61,11.97,2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
266,VAMLBDMF1,7.9,8.0,7.9,0.0,7.8,0.1,2,0.008,1002,Mutual Funds,A,1.50,0.44,2.13,14.63,2021
267,VAMLRBBF,7.5,7.6,7.5,0.0,7.6,-0.1,22,0.266,35445,Mutual Funds,A,0.16,1.30,1.68,10.16,2020
268,VFSTDL,25.8,26.4,24.5,0.0,25.0,0.8,1796,163.966,6529179,Textile,A,1.10,1.76,1.50,18.70,2021
269,WALTONHIL,1086.0,1090.0,1081.1,0.0,1085.8,0.2,11,0.043,40,Engineering,A,25.00,29.46,54.23,311.59,2021


In [23]:
df_for_lin_reg = df_merged_mp_disclosures.loc[:, ['TRADING CODE', 'LTP*', 'HIGH', 'LOW', 'DIVIDEND', 'EPS_Q', 'EPS_Y', 'NAVPS','SECTOR']]

In [24]:
df_for_lin_reg

,TRADING CODE,LTP*,HIGH,LOW,DIVIDEND,EPS_Q,EPS_Y,NAVPS,SECTOR
0,1JANATAMF,6.5,6.6,6.5,1.30,0.78,2.54,11.94,Mutual Funds
1,1STPRIMFMF,18.2,18.4,18.0,0.80,1.40,0.38,11.62,Mutual Funds
2,AAMRANET,47.2,47.9,47.2,2.65,1.88,2.14,37.09,IT Sector
3,AAMRATECH,38.8,39.7,38.7,2.21,2.32,1.45,23.93,IT Sector
4,ABB1STMF,5.8,5.8,5.8,0.80,0.92,2.61,11.97,Mutual Funds
...,...,...,...,...,...,...,...,...,...
266,VAMLBDMF1,7.9,8.0,7.9,1.50,0.44,2.13,14.63,Mutual Funds
267,VAMLRBBF,7.5,7.6,7.5,0.16,1.30,1.68,10.16,Mutual Funds
268,VFSTDL,25.8,26.4,24.5,1.10,1.76,1.50,18.70,Textile
269,WALTONHIL,1086.0,1090.0,1081.1,25.00,29.46,54.23,311.59,Engineering


In [25]:
def div_yield(row):
    return round((row['DIVIDEND'] / row['LTP*']) * 100, 2)

In [26]:
def pe_q(row):
    return round(row['LTP*'] / row['EPS_Q'], 2)

In [27]:
def pe_y(row):
    return round(row['LTP*'] / row['EPS_Y'], 2)

In [28]:
def mp_to_nav(row):
    return round(((row['NAVPS'] - row['LTP*']) / row['LTP*']) * 100 ,2)

In [29]:
def mp_to_weigthed_average(row):
    return round(((row['LTP*'] - ((row['HIGH'] * .4) + (row['LOW']) * .6)) / row['LTP*']) * 100 ,2)

In [30]:
def roe_q(row):
    return round((row['EPS_Q'] / row['NAVPS']) * 100, 2)

In [31]:
def roe_y(row):
    return round((row['EPS_Y'] / row['NAVPS']) * 100, 2)

In [32]:
df_for_lin_reg['YIELD'] = df_for_lin_reg.apply(lambda row: div_yield(row), axis = 1)
df_for_lin_reg['pe_q'] = df_for_lin_reg.apply(lambda row: pe_q(row), axis = 1)
df_for_lin_reg['pe_y'] = df_for_lin_reg.apply(lambda row: pe_y(row), axis = 1)
df_for_lin_reg['NAV>MP'] = df_for_lin_reg.apply(lambda row: mp_to_nav(row), axis = 1)
df_for_lin_reg['MP-WMP'] = df_for_lin_reg.apply(lambda row: mp_to_weigthed_average(row), axis = 1)
df_for_lin_reg['ROE_Q'] = df_for_lin_reg.apply(lambda row: roe_q(row), axis = 1)
df_for_lin_reg['ROE_Y'] = df_for_lin_reg.apply(lambda row: roe_y(row), axis = 1)

In [33]:
sectors, frequencies = np.unique(df_for_lin_reg['SECTOR'], return_counts = True)

In [34]:
from sklearn.linear_model import LinearRegression

In [35]:
from sklearn.metrics import r2_score

In [36]:
import statsmodels.api as sm

In [37]:
data = df_for_lin_reg
X = np.column_stack((data['DIVIDEND'], data['NAVPS'], data['EPS_Q'], data['EPS_Y']))
y = data['LTP*']
X2 = sm.add_constant(X)
est = sm.OLS(y, X2)
est2 = est.fit()
print(est2.summary())
print('\n\n\n\n')

for sector, frequency in zip(sectors, frequencies):

    if frequency > 10:
        data = df_for_lin_reg.groupby('SECTOR').get_group(sector)
        X = np.column_stack((data['DIVIDEND'], data['NAVPS'], data['EPS_Q'], data['EPS_Y']))
        y = data['LTP*']
        X2 = sm.add_constant(X)
        est = sm.OLS(y, X2)
        est2 = est.fit()
        print(f'{sector}')
        print(est2.summary())
        print('\n\n\n\n')
        
    elif frequency < 11:
        continue

                            OLS Regression Results                            
Dep. Variable:                   LTP*   R-squared:                       0.776
Model:                            OLS   Adj. R-squared:                  0.772
Method:                 Least Squares   F-statistic:                     229.9
Date:                Tue, 22 Mar 2022   Prob (F-statistic):           4.87e-85
Time:                        11:39:32   Log-Likelihood:                -1835.4
No. Observations:                 271   AIC:                             3681.
Df Residuals:                     266   BIC:                             3699.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        -27.9951     14.995     -1.867      0.0

/home/ismail/anaconda3/lib/python3.7/site-packages/scipy/stats/stats.py:1542: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=11
  "anyway, n=%i" % int(n))


In [38]:
sector_table_dict = dict()
count = 0

for sector, frequency in zip(sectors, frequencies):
    if frequency > 10:
        sector_table_dict[count] = f'df_for_lin_reg.groupby(\'SECTOR\').get_group(\'{sector}\')'
        count += 1
del count

In [39]:
df_for_lin_reg.groupby('SECTOR').get_group('Bank').loc[:, ['TRADING CODE', 'LTP*', 'HIGH', 'LOW', 'DIVIDEND', 'EPS_Q', 'EPS_Y', 'NAVPS', 'ROE_Q', 'ROE_Y', 'YIELD', 'pe_q', 'pe_y', 'NAV>MP', 'MP-WMP']]

,TRADING CODE,LTP*,HIGH,LOW,DIVIDEND,EPS_Q,EPS_Y,NAVPS,ROE_Q,ROE_Y,YIELD,pe_q,pe_y,NAV>MP,MP-WMP
5,ABBANK,12.4,12.5,12.3,0.64,0.59,0.50,32.26,1.83,1.55,5.16,21.02,24.80,160.16,0.16
18,ALARABANK,26.5,26.5,26.5,1.50,2.05,2.41,22.35,9.17,10.78,5.66,12.93,11.00,-15.66,0.00
33,BANKASIA,21.1,21.2,21.0,1.00,3.18,1.76,23.34,13.62,7.54,4.74,6.64,11.99,10.62,0.09
49,BRACBANK,49.5,49.7,49.4,3.76,3.83,3.33,36.63,10.46,9.09,7.60,12.92,14.86,-26.00,-0.04
58,CITYBANK,26.4,26.8,26.3,3.09,3.96,4.29,29.03,13.64,14.78,11.70,6.67,6.15,9.96,-0.38
70,DHAKABANK,14.4,14.6,14.2,1.46,2.20,2.22,21.69,10.14,10.24,10.14,6.55,6.49,50.63,0.28
76,DUTCHBANGL,76.9,77.4,76.9,13.20,8.33,10.00,58.65,14.20,17.05,17.17,9.23,7.69,-23.73,-0.26
80,EBL,38.5,38.5,38.4,8.42,5.61,5.15,36.28,15.46,14.20,21.87,6.86,7.48,-5.77,0.16
91,EXIMBANK,12.5,12.6,12.4,1.06,1.57,1.99,21.66,7.25,9.19,8.48,7.96,6.28,73.28,0.16
95,FIRSTSBANK,12.0,12.0,11.9,1.12,1.73,2.93,18.91,9.15,15.49,9.33,6.94,4.10,57.58,0.50


In [40]:
df_for_lin_reg.groupby('SECTOR').get_group('Engineering').loc[:, ['TRADING CODE', 'LTP*', 'HIGH', 'LOW', 'DIVIDEND', 'EPS_Q', 'EPS_Y', 'NAVPS', 'ROE_Q', 'ROE_Y', 'YIELD', 'pe_q', 'pe_y', 'NAV>MP', 'MP-WMP']]

,TRADING CODE,LTP*,HIGH,LOW,DIVIDEND,EPS_Q,EPS_Y,NAVPS,ROE_Q,ROE_Y,YIELD,pe_q,pe_y,NAV>MP,MP-WMP
22,ANWARGALV,380.0,383.0,366.0,22.73,15.26,2.03,10.85,140.65,18.71,5.98,24.90,187.19,-97.14,1.89
37,BBSCABLES,56.8,57.2,56.1,3.95,5.04,4.84,33.50,15.04,14.45,6.95,11.27,11.74,-41.02,0.46
38,BDAUTOCA,144.2,144.4,142.0,0.40,0.52,0.37,6.85,7.59,5.40,0.28,277.31,389.73,-95.25,0.86
41,BDLAMPS,311.9,313.0,300.0,2.00,8.84,5.10,95.16,9.29,5.36,0.64,35.28,61.16,-69.49,2.15
42,BDTHAI,19.4,19.6,19.3,0.20,0.26,1.25,28.11,0.92,4.45,1.03,74.62,15.52,44.90,-0.10
44,BENGALWTL,23.7,23.9,23.4,0.50,0.58,0.50,24.87,2.33,2.01,2.11,40.86,47.40,4.94,0.42
52,BSRMLTD,104.0,104.5,103.0,5.00,16.22,16.66,127.56,12.72,13.06,4.81,6.41,6.24,22.65,0.38
53,BSRMSTEEL,73.0,73.5,73.0,4.00,10.69,8.10,62.38,17.14,12.98,5.48,6.83,9.01,-14.55,-0.27
62,COPPERTECH,38.2,38.2,37.6,0.25,1.64,0.35,11.24,14.59,3.11,0.65,23.29,109.14,-70.58,0.94
68,DESHBANDHU,21.5,21.8,21.5,0.50,0.66,0.20,18.07,3.65,1.11,2.33,32.58,107.50,-15.95,-0.56


In [41]:
df_for_lin_reg.groupby('SECTOR').get_group('Financial Institutions').loc[:, ['TRADING CODE', 'LTP*', 'HIGH', 'LOW', 'DIVIDEND', 'EPS_Q', 'EPS_Y', 'NAVPS', 'ROE_Q', 'ROE_Y', 'YIELD', 'pe_q', 'pe_y', 'NAV>MP', 'MP-WMP']]

,TRADING CODE,LTP*,HIGH,LOW,DIVIDEND,EPS_Q,EPS_Y,NAVPS,ROE_Q,ROE_Y,YIELD,pe_q,pe_y,NAV>MP,MP-WMP
36,BAYLEASING,28.0,28.4,27.8,1.00,3.67,1.14,19.80,18.54,5.76,3.57,7.63,24.56,-29.29,-0.14
40,BDFINANCE,39.9,40.9,39.5,3.43,1.44,1.80,17.86,8.06,10.08,8.60,27.71,22.17,-55.24,-0.40
108,GSPFINANCE,27.9,28.1,27.4,2.49,1.69,1.55,21.90,7.72,7.08,8.92,16.51,18.00,-21.51,0.79
113,ICB,108.0,108.6,106.7,1.10,3.48,1.43,54.98,6.33,2.60,1.02,31.03,75.52,-49.09,0.50
118,IDLC,51.2,51.4,51.2,4.31,5.34,6.74,40.41,13.21,16.68,8.42,9.59,7.60,-21.07,-0.16
127,ISLAMICFIN,23.7,23.8,23.3,1.00,1.19,1.54,14.99,7.94,10.27,4.22,19.92,15.39,-36.75,0.84
140,LANKABAFIN,33.3,33.6,32.9,1.20,2.23,1.81,18.73,11.91,9.66,3.60,14.93,18.40,-43.75,0.36
170,NHFIL,45.0,45.5,44.9,1.50,2.81,2.75,17.91,15.69,15.35,3.33,16.01,16.36,-60.20,-0.31
188,PHOENIXFIN,21.2,21.9,21.2,2.92,0.97,1.36,22.01,4.41,6.18,13.77,21.86,15.59,3.82,-1.32
261,UNITEDFIN,18.9,18.9,18.7,1.00,0.96,1.14,17.11,5.61,6.66,5.29,19.69,16.58,-9.47,0.63


In [42]:
df_for_lin_reg.groupby('SECTOR').get_group('Food & Allied').loc[:, ['TRADING CODE', 'LTP*', 'HIGH', 'LOW', 'DIVIDEND', 'EPS_Q', 'EPS_Y', 'NAVPS', 'ROE_Q', 'ROE_Y', 'YIELD', 'pe_q', 'pe_y', 'NAV>MP', 'MP-WMP']]

,TRADING CODE,LTP*,HIGH,LOW,DIVIDEND,EPS_Q,EPS_Y,NAVPS,ROE_Q,ROE_Y,YIELD,pe_q,pe_y,NAV>MP,MP-WMP
21,AMCL(PRAN),336.8,336.9,328.9,3.20,8.70,5.37,84.17,10.34,6.38,0.95,38.71,62.72,-75.01,1.40
24,APEXFOODS,224.8,224.8,207.1,2.00,2.20,2.07,119.39,1.84,1.73,0.89,102.18,108.60,-46.89,4.72
32,BANGAS,125.7,125.7,123.3,0.40,0.18,0.23,21.02,0.86,1.09,0.32,698.33,546.52,-83.28,1.15
35,BATBC,587.0,590.0,585.2,1292.60,27.72,60.48,188.89,14.68,32.02,220.20,21.18,9.71,-67.82,-0.02
86,EMERALDOIL,34.8,35.6,34.8,3.49,0.12,3.33,17.23,0.70,19.33,10.03,290.00,10.45,-50.49,-0.92
99,GEMINISEA,470.0,474.7,445.0,0.50,10.24,0.72,0.93,1101.08,77.42,0.11,45.90,652.78,-99.80,2.79
145,LOVELLO,37.3,37.7,37.0,1.10,1.10,1.41,12.57,8.75,11.22,2.95,33.91,26.45,-66.30,0.05
175,OLYMPIC,138.8,139.5,137.5,5.40,7.22,10.19,46.21,15.62,22.05,3.89,19.22,13.62,-66.71,0.36
209,RDFOOD,58.7,59.5,57.1,2.13,1.54,0.64,14.65,10.51,4.37,3.63,38.12,91.72,-75.04,1.09
260,UNILEVERCL,2880.0,2885.0,2870.0,44.00,42.80,43.94,123.08,34.77,35.70,1.53,67.29,65.54,-95.73,0.14


In [43]:
df_for_lin_reg.groupby('SECTOR').get_group('Fuel & Power').loc[:, ['TRADING CODE', 'LTP*', 'HIGH', 'LOW', 'DIVIDEND', 'EPS_Q', 'EPS_Y', 'NAVPS', 'ROE_Q', 'ROE_Y', 'YIELD', 'pe_q', 'pe_y', 'NAV>MP', 'MP-WMP']]

,TRADING CODE,LTP*,HIGH,LOW,DIVIDEND,EPS_Q,EPS_Y,NAVPS,ROE_Q,ROE_Y,YIELD,pe_q,pe_y,NAV>MP,MP-WMP
23,AOL,41.8,42.3,41.5,3.60,1.88,1.58,19.25,9.77,8.21,8.61,22.23,26.46,-53.95,-0.05
34,BARKAPOWER,24.3,24.5,24.2,1.00,3.38,2.96,20.91,16.16,14.16,4.12,7.19,8.21,-13.95,-0.08
48,BPPL,33.2,33.4,33.0,1.25,3.72,3.71,16.34,22.77,22.71,3.77,8.92,8.95,-50.78,0.12
67,DESCO,38.4,38.5,37.4,1.00,1.02,1.15,46.76,2.18,2.46,2.60,37.65,33.39,21.77,1.46
73,DOREENPWR,70.1,70.2,69.8,10.07,9.75,8.11,48.41,20.14,16.75,14.37,7.19,8.64,-30.94,0.20
79,EASTRNLUB,2146.0,2154.0,2135.0,3.00,5.92,5.43,178.45,3.32,3.04,0.14,362.50,395.21,-91.68,0.16
88,EPGL,39.8,39.9,39.5,1.00,2.02,2.02,57.72,3.50,3.50,2.51,19.70,19.70,45.03,0.35
98,GBBPOWER,35.5,36.2,35.5,1.15,1.20,1.50,20.93,5.73,7.17,3.24,29.58,23.67,-41.04,-0.79
125,INTRACO,21.6,21.7,21.3,1.58,1.34,0.55,12.28,10.91,4.48,7.31,16.12,39.27,-43.15,0.65
131,JAMUNAOIL,161.9,163.7,161.9,12.00,16.86,18.13,161.40,10.45,11.23,7.41,9.60,8.93,-0.31,-0.44


In [44]:
df_for_lin_reg.groupby('SECTOR').get_group('Insurance').loc[:, ['TRADING CODE', 'LTP*', 'HIGH', 'LOW', 'DIVIDEND', 'EPS_Q', 'EPS_Y', 'NAVPS', 'ROE_Q', 'ROE_Y', 'YIELD', 'pe_q', 'pe_y', 'NAV>MP', 'MP-WMP']]

,TRADING CODE,LTP*,HIGH,LOW,DIVIDEND,EPS_Q,EPS_Y,NAVPS,ROE_Q,ROE_Y,YIELD,pe_q,pe_y,NAV>MP,MP-WMP
15,AGRANINS,47.3,48.0,47.1,3.15,1.83,1.72,18.11,10.10,9.50,6.66,25.85,27.50,-61.71,-0.34
29,ASIAINS,78.2,79.6,77.7,1.20,4.24,2.34,24.52,17.29,9.54,1.53,18.44,33.42,-68.64,-0.33
30,ASIAPACINS,52.8,52.8,52.5,1.00,3.33,1.97,20.63,16.14,9.55,1.89,15.86,26.80,-60.93,0.34
46,BGIC,47.7,50.9,47.7,1.20,2.56,1.28,19.38,13.21,6.60,2.52,18.63,37.27,-59.37,-2.68
57,CENTRALINS,45.9,46.3,45.0,4.32,2.68,2.18,25.02,10.71,8.71,9.41,17.13,21.06,-45.49,0.83
61,CONTININS,38.3,39.4,37.9,2.40,2.27,1.29,19.84,11.44,6.50,6.27,16.87,29.69,-48.20,-0.52
63,CRYSTALINS,43.6,43.6,43.0,1.00,2.45,4.58,20.98,11.68,21.83,2.29,17.80,9.52,-51.88,0.83
69,DGIC,34.0,34.1,34.0,1.00,1.32,0.91,7.16,18.44,12.71,2.94,25.76,37.36,-78.94,-0.12
71,DHAKAINS,60.9,61.0,60.9,2.00,3.31,2.89,31.61,10.47,9.14,3.28,18.40,21.07,-48.10,-0.07
77,EASTERNINS,77.7,79.0,77.5,2.00,5.39,3.35,47.33,11.39,7.08,2.57,14.42,23.19,-39.09,-0.51


In [45]:
df_for_lin_reg.groupby('SECTOR').get_group('Mutual Funds').loc[:, ['TRADING CODE', 'LTP*', 'HIGH', 'LOW', 'DIVIDEND', 'EPS_Q', 'EPS_Y', 'NAVPS', 'ROE_Q', 'ROE_Y', 'YIELD', 'pe_q', 'pe_y', 'NAV>MP', 'MP-WMP']]

,TRADING CODE,LTP*,HIGH,LOW,DIVIDEND,EPS_Q,EPS_Y,NAVPS,ROE_Q,ROE_Y,YIELD,pe_q,pe_y,NAV>MP,MP-WMP
0,1JANATAMF,6.5,6.6,6.5,1.30,0.78,2.54,11.94,6.53,21.27,20.00,8.33,2.56,83.69,-0.62
1,1STPRIMFMF,18.2,18.4,18.0,0.80,1.40,0.38,11.62,12.05,3.27,4.40,13.00,47.89,-36.15,0.22
4,ABB1STMF,5.8,5.8,5.8,0.80,0.92,2.61,11.97,7.69,21.80,13.79,6.30,2.22,106.38,0.00
16,AIBL1STIMF,8.8,8.8,8.8,1.23,0.53,1.85,11.57,4.58,15.99,13.98,16.60,4.76,31.48,0.00
31,ATCSLGF,10.1,10.1,10.0,1.50,1.42,1.93,12.41,11.44,15.55,14.85,7.11,5.23,22.87,0.59
55,CAPMBDBLMF,9.8,10.2,9.8,1.30,0.84,2.41,11.81,7.11,20.41,13.27,11.67,4.07,20.51,-1.63
56,CAPMIBBLMF,15.7,16.0,15.7,1.35,0.74,1.89,11.97,6.18,15.79,8.60,21.22,8.31,-23.76,-0.76
66,DBH1STMF,7.5,7.5,7.5,1.20,0.13,2.13,11.79,1.10,18.07,16.00,57.69,3.52,57.20,0.00
81,EBL1STMF,7.6,7.6,7.6,1.30,1.10,2.61,11.76,9.35,22.19,17.11,6.91,2.91,54.74,0.00
82,EBLNRBMF,6.5,6.6,6.5,0.60,1.66,1.92,12.27,13.53,15.65,9.23,3.92,3.39,88.77,-0.62


In [46]:
df_for_lin_reg.groupby('SECTOR').get_group('Pharmaceuticals & Chemicals').loc[:, ['TRADING CODE', 'LTP*', 'HIGH', 'LOW', 'DIVIDEND', 'EPS_Q', 'EPS_Y', 'NAVPS', 'ROE_Q', 'ROE_Y', 'YIELD', 'pe_q', 'pe_y', 'NAV>MP', 'MP-WMP']]

,TRADING CODE,LTP*,HIGH,LOW,DIVIDEND,EPS_Q,EPS_Y,NAVPS,ROE_Q,ROE_Y,YIELD,pe_q,pe_y,NAV>MP,MP-WMP
7,ACI,290.0,290.0,286.0,50.66,11.02,5.50,162.65,6.78,3.38,17.47,26.32,52.73,-43.91,0.83
8,ACIFORMULA,171.0,171.0,169.0,11.60,7.42,4.74,66.77,11.11,7.10,6.78,23.05,36.08,-60.95,0.70
9,ACMELAB,90.3,90.8,89.0,2.50,10.34,7.42,95.04,10.88,7.81,2.77,8.73,12.17,5.25,0.64
10,ACTIVEFINE,23.4,23.6,23.1,0.05,0.42,0.16,22.01,1.91,0.73,0.21,55.71,146.25,-5.94,0.43
12,ADVENT,29.9,30.2,29.6,2.50,1.36,1.41,13.50,10.07,10.44,8.36,21.99,21.21,-54.85,0.20
13,AFCAGRO,25.1,26.1,25.1,0.05,0.66,0.15,18.00,3.67,0.83,0.20,38.03,167.33,-28.29,-1.59
43,BEACONPHAR,230.8,231.5,227.9,1.50,5.19,3.75,23.18,22.39,16.18,0.65,44.47,61.55,-89.96,0.63
54,BXPHARMA,180.3,181.0,179.3,3.50,14.24,11.49,83.01,17.15,13.84,1.94,12.66,15.69,-53.96,0.18
111,IBNSINA,288.0,292.0,288.0,4.70,21.03,15.69,68.69,30.62,22.84,1.63,13.69,18.36,-76.15,-0.56
112,IBP,19.2,19.4,18.9,0.81,1.02,1.40,13.61,7.49,10.29,4.22,18.82,13.71,-29.11,0.52


In [47]:
df_for_lin_reg.groupby('SECTOR').get_group('Textile').loc[:, ['TRADING CODE', 'LTP*', 'HIGH', 'LOW', 'DIVIDEND', 'EPS_Q', 'EPS_Y', 'NAVPS', 'ROE_Q', 'ROE_Y', 'YIELD', 'pe_q', 'pe_y', 'NAV>MP', 'MP-WMP']]

,TRADING CODE,LTP*,HIGH,LOW,DIVIDEND,EPS_Q,EPS_Y,NAVPS,ROE_Q,ROE_Y,YIELD,pe_q,pe_y,NAV>MP,MP-WMP
6,ACFL,28.9,29.1,28.6,1.10,1.10,1.25,34.72,3.17,3.60,3.81,26.27,23.12,20.14,0.35
17,AIL,48.3,48.8,47.6,1.00,1.50,1.45,21.66,6.93,6.69,2.07,32.20,33.31,-55.16,0.46
19,ALIF,14.4,14.5,14.3,0.20,0.64,0.44,14.18,4.51,3.10,1.39,22.50,32.73,-1.53,0.14
26,APEXSPINN,166.6,166.7,161.5,2.00,3.62,2.93,55.81,6.49,5.25,1.20,46.02,56.86,-66.50,1.81
59,CNATEX,10.5,10.7,10.4,0.92,0.72,1.14,16.74,4.30,6.81,8.76,14.58,9.21,59.43,-0.19
64,DACCADYE,22.2,22.5,22.0,0.10,1.46,0.13,8.36,17.46,1.56,0.45,15.21,170.77,-62.34,0.00
74,DSHGARME,167.0,167.0,161.3,0.50,1.08,0.40,19.70,5.48,2.03,0.30,154.63,417.50,-88.20,2.05
75,DSSL,22.3,22.6,22.3,2.37,1.32,1.23,18.98,6.95,6.48,10.63,16.89,18.13,-14.89,-0.54
87,ENVOYTEX,47.2,47.2,47.2,0.50,2.72,0.56,37.79,7.20,1.48,1.06,17.35,84.29,-19.94,0.00
89,ESQUIRENIT,34.5,34.7,34.3,1.50,2.74,1.97,50.61,5.41,3.89,4.35,12.59,17.51,46.70,0.12


In [48]:
sector_table_dict

{0: "df_for_lin_reg.groupby('SECTOR').get_group('Bank')",
 1: "df_for_lin_reg.groupby('SECTOR').get_group('Engineering')",
 2: "df_for_lin_reg.groupby('SECTOR').get_group('Financial Institutions')",
 3: "df_for_lin_reg.groupby('SECTOR').get_group('Fuel & Power')",
 4: "df_for_lin_reg.groupby('SECTOR').get_group('Insurance')",
 5: "df_for_lin_reg.groupby('SECTOR').get_group('Mutual Funds')",
 6: "df_for_lin_reg.groupby('SECTOR').get_group('Pharmaceuticals & Chemicals')",
 7: "df_for_lin_reg.groupby('SECTOR').get_group('Textile')"}

In [50]:
data_banks = df_for_lin_reg.groupby('SECTOR').get_group('Bank')

In [92]:
X = np.column_stack((data_banks['DIVIDEND'], data_banks['NAVPS'], data_banks['EPS_Q'], data_banks['EPS_Y']))

In [93]:
y = data_banks['LTP*']

In [94]:
from sklearn.linear_model import LinearRegression

In [95]:
from sklearn.metrics import r2_score

In [96]:
import statsmodels.api as sm

In [97]:
X2 = sm.add_constant(X)

In [98]:
est = sm.OLS(y, X2)

In [99]:
est2 = est.fit()

In [100]:
print(est2.summary())

                            OLS Regression Results                            
Dep. Variable:                   LTP*   R-squared:                       0.810
Model:                            OLS   Adj. R-squared:                  0.781
Method:                 Least Squares   F-statistic:                     27.76
Date:                Sun, 30 Jan 2022   Prob (F-statistic):           4.76e-09
Time:                        20:38:56   Log-Likelihood:                -99.850
No. Observations:                  31   AIC:                             209.7
Df Residuals:                      26   BIC:                             216.9
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.1647      3.931      0.042      0.9

In [101]:
data_mf = df_for_lin_reg.groupby('SECTOR').get_group('Mutual Funds')

In [57]:
data_mf

,TRADING CODE,LTP*,DIVIDEND,EPS_Q,EPS_Y,NAVPS,SECTOR,YIELD,pe_q,pe_y,NAV>MP
0,1JANATAMF,6.5,1.30,0.80,2.54,11.94,Mutual Funds,20.00,8.12,2.56,83.69
1,1STPRIMFMF,19.4,0.80,1.48,0.38,11.62,Mutual Funds,4.12,13.11,51.05,-40.10
4,ABB1STMF,5.6,0.80,0.68,2.61,11.97,Mutual Funds,14.29,8.24,2.15,113.75
16,AIBL1STIMF,8.9,1.23,0.21,1.83,11.57,Mutual Funds,13.82,42.38,4.86,30.00
31,ATCSLGF,9.9,1.50,1.12,1.93,12.41,Mutual Funds,15.15,8.84,5.13,25.35
57,CAPMBDBLMF,10.1,1.30,1.48,2.41,11.81,Mutual Funds,12.87,6.82,4.19,16.93
58,CAPMIBBLMF,15.7,1.35,1.28,1.89,11.97,Mutual Funds,8.60,12.27,8.31,-23.76
70,DBH1STMF,7.6,1.20,0.13,2.13,11.79,Mutual Funds,15.79,58.46,3.57,55.13
84,EBL1STMF,7.5,1.30,2.13,2.58,11.76,Mutual Funds,17.33,3.52,2.91,56.80
85,EBLNRBMF,6.1,0.60,1.16,1.92,12.27,Mutual Funds,9.84,5.26,3.18,101.15


In [58]:
X_mf = np.column_stack((data_mf['DIVIDEND'], data_mf['NAVPS'], data_mf['EPS_Q'], data_mf['EPS_Y']))

In [59]:
y_mf = data_mf['LTP*']

In [60]:
X2_mf = sm.add_constant(X_mf)

In [61]:
est_mf = sm.OLS(y_mf, X2_mf)

In [62]:
est2_mf = est_mf.fit()

In [63]:
print(est2_mf.summary())

                            OLS Regression Results                            
Dep. Variable:                   LTP*   R-squared:                       0.421
Model:                            OLS   Adj. R-squared:                  0.346
Method:                 Least Squares   F-statistic:                     5.637
Date:                Wed, 26 Jan 2022   Prob (F-statistic):            0.00157
Time:                        20:36:18   Log-Likelihood:                -82.400
No. Observations:                  36   AIC:                             174.8
Df Residuals:                      31   BIC:                             182.7
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.0890      2.627      0.415      0.6

In [64]:
data_ins = df_for_lin_reg.groupby('SECTOR').get_group('Insurance')

In [65]:
X_ins = np.column_stack((data_ins['DIVIDEND'], data_ins['NAVPS'], data_ins['EPS_Q'], data_ins['EPS_Y']))

In [66]:
y_ins = data_ins['LTP*']

In [67]:
X2_ins = sm.add_constant(X_ins)

In [68]:
est_ins = sm.OLS(y_ins, X2_ins)

In [69]:
est2_ins = est_ins.fit()

In [70]:
print(est2_ins.summary())

                            OLS Regression Results                            
Dep. Variable:                   LTP*   R-squared:                       0.759
Model:                            OLS   Adj. R-squared:                  0.729
Method:                 Least Squares   F-statistic:                     25.24
Date:                Wed, 26 Jan 2022   Prob (F-statistic):           1.66e-09
Time:                        20:36:19   Log-Likelihood:                -141.85
No. Observations:                  37   AIC:                             293.7
Df Residuals:                      32   BIC:                             301.8
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         33.9246      4.248      7.986      0.0

TEXTILE

In [71]:
data_tex = df_for_lin_reg.groupby('SECTOR').get_group('Textile')
X_tex = np.column_stack((data_tex['DIVIDEND'], data_tex['NAVPS'], data_tex['EPS_Q'], data_tex['EPS_Y']))
y_tex = data_tex['LTP*']
X2_tex = sm.add_constant(X_tex)
est_tex = sm.OLS(y_tex, X2_tex)
est2_tex = est_tex.fit()
print(est2_tex.summary())

                            OLS Regression Results                            
Dep. Variable:                   LTP*   R-squared:                       0.208
Model:                            OLS   Adj. R-squared:                  0.098
Method:                 Least Squares   F-statistic:                     1.901
Date:                Wed, 26 Jan 2022   Prob (F-statistic):              0.137
Time:                        20:36:19   Log-Likelihood:                -180.80
No. Observations:                  34   AIC:                             371.6
Df Residuals:                      29   BIC:                             379.2
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          2.4980     20.780      0.120      0.9

In [72]:
#Engineering

In [73]:
data_engn = df_for_lin_reg.groupby('SECTOR').get_group('Engineering')
X_engn = np.column_stack((data_engn['DIVIDEND'], data_engn['NAVPS'], data_engn['EPS_Q'], data_engn['EPS_Y']))
y_engn = data_engn['LTP*']
X2_engn = sm.add_constant(X_engn)
est_engn = sm.OLS(y_engn, X2_engn)
est2_engn = est_engn.fit()
print(est2_engn.summary())

                            OLS Regression Results                            
Dep. Variable:                   LTP*   R-squared:                       0.701
Model:                            OLS   Adj. R-squared:                  0.651
Method:                 Least Squares   F-statistic:                     14.06
Date:                Wed, 26 Jan 2022   Prob (F-statistic):           4.81e-06
Time:                        20:36:19   Log-Likelihood:                -179.61
No. Observations:                  29   AIC:                             369.2
Df Residuals:                      24   BIC:                             376.0
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         31.6004     41.557      0.760      0.4

In [74]:
data_fuel = df_for_lin_reg.groupby('SECTOR').get_group('Fuel & Power')
X_fuel = np.column_stack((data_fuel['DIVIDEND'], data_fuel['NAVPS'], data_fuel['EPS_Q'], data_fuel['EPS_Y']))
y_fuel = data_fuel['LTP*']
X2_fuel = sm.add_constant(X_fuel)
est_fuel = sm.OLS(y_fuel, X2_fuel)
est2_fuel = est_fuel.fit()
print(est2_fuel.summary())

                            OLS Regression Results                            
Dep. Variable:                   LTP*   R-squared:                       0.710
Model:                            OLS   Adj. R-squared:                  0.627
Method:                 Least Squares   F-statistic:                     8.569
Date:                Wed, 26 Jan 2022   Prob (F-statistic):            0.00103
Time:                        20:36:19   Log-Likelihood:                -138.20
No. Observations:                  19   AIC:                             286.4
Df Residuals:                      14   BIC:                             291.1
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       -237.9085    157.125     -1.514      0.1

/home/ismail/anaconda3/lib/python3.7/site-packages/scipy/stats/stats.py:1542: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=19
  "anyway, n=%i" % int(n))


In [75]:
data_fuel = df_for_lin_reg.groupby('SECTOR').get_group('Miscellaneous')
X_fuel = np.column_stack((data_fuel['DIVIDEND'], data_fuel['NAVPS'], data_fuel['EPS_Q'], data_fuel['EPS_Y']))
y_fuel = data_fuel['LTP*']
X2_fuel = sm.add_constant(X_fuel)
est_fuel = sm.OLS(y_fuel, X2_fuel)
est2_fuel = est_fuel.fit()
print(est2_fuel.summary())

                            OLS Regression Results                            
Dep. Variable:                   LTP*   R-squared:                       0.997
Model:                            OLS   Adj. R-squared:                  0.994
Method:                 Least Squares   F-statistic:                     322.1
Date:                Wed, 26 Jan 2022   Prob (F-statistic):           2.87e-05
Time:                        20:36:19   Log-Likelihood:                -43.345
No. Observations:                   9   AIC:                             96.69
Df Residuals:                       4   BIC:                             97.68
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        -29.9607     25.515     -1.174      0.3

/home/ismail/anaconda3/lib/python3.7/site-packages/scipy/stats/stats.py:1542: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=9
  "anyway, n=%i" % int(n))


In [76]:
sectors = df_for_lin_reg['SECTOR'].unique()

In [77]:
sectors

array(['Mutual Funds', 'IT Sector', 'Bank', 'Textile',
       'Pharmaceuticals & Chemicals', 'Insurance', 'Miscellaneous',
       'Food & Allied', 'Engineering', 'Fuel & Power',
       'Tannery Industries', 'Financial Institutions', 'Paper & Printing',
       'Telecommunication', 'Cement', 'Services & Real Estate',
       'Ceramics Sector', 'Travel & Leisure', 'Jute'], dtype=object)

In [78]:
for sector in sectors:
    data_fuel = df_for_lin_reg.groupby('SECTOR').get_group(sector)
    X_fuel = np.column_stack((data_fuel['DIVIDEND'], data_fuel['NAVPS'], data_fuel['EPS_Q'], data_fuel['EPS_Y']))
    y_fuel = data_fuel['LTP*']
    X2_fuel = sm.add_constant(X_fuel)
    est_fuel = sm.OLS(y_fuel, X2_fuel)
    est2_fuel = est_fuel.fit()
    print(f'{sector}')
    print(est2_fuel.summary())
    print('\n\n\n')

Mutual Funds
                            OLS Regression Results                            
Dep. Variable:                   LTP*   R-squared:                       0.421
Model:                            OLS   Adj. R-squared:                  0.346
Method:                 Least Squares   F-statistic:                     5.637
Date:                Wed, 26 Jan 2022   Prob (F-statistic):            0.00157
Time:                        20:36:19   Log-Likelihood:                -82.400
No. Observations:                  36   AIC:                             174.8
Df Residuals:                      31   BIC:                             182.7
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.0890      2.627      0

/home/ismail/anaconda3/lib/python3.7/site-packages/scipy/stats/stats.py:1542: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=10
  "anyway, n=%i" % int(n))
/home/ismail/anaconda3/lib/python3.7/site-packages/scipy/stats/stats.py:1542: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=9
  "anyway, n=%i" % int(n))
/home/ismail/anaconda3/lib/python3.7/site-packages/scipy/stats/stats.py:1542: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=11
  "anyway, n=%i" % int(n))


                            OLS Regression Results                            
Dep. Variable:                   LTP*   R-squared:                       0.981
Model:                            OLS   Adj. R-squared:                  0.977
Method:                 Least Squares   F-statistic:                     259.0
Date:                Wed, 26 Jan 2022   Prob (F-statistic):           6.42e-17
Time:                        20:36:19   Log-Likelihood:                -163.23
No. Observations:                  25   AIC:                             336.5
Df Residuals:                      20   BIC:                             342.6
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          6.8373     46.239      0.148      0.8

/home/ismail/anaconda3/lib/python3.7/site-packages/scipy/stats/stats.py:1542: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=19
  "anyway, n=%i" % int(n))


                            OLS Regression Results                            
Dep. Variable:                   LTP*   R-squared:                       0.701
Model:                            OLS   Adj. R-squared:                  0.651
Method:                 Least Squares   F-statistic:                     14.06
Date:                Wed, 26 Jan 2022   Prob (F-statistic):           4.81e-06
Time:                        20:36:19   Log-Likelihood:                -179.61
No. Observations:                  29   AIC:                             369.2
Df Residuals:                      24   BIC:                             376.0
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         31.6004     41.557      0.760      0.4

/home/ismail/anaconda3/lib/python3.7/site-packages/statsmodels/stats/stattools.py:75: ValueWarning: omni_normtest is not valid with less than 8 observations; 4 samples were given.
  "samples were given." % int(n), ValueWarning)
/home/ismail/anaconda3/lib/python3.7/site-packages/statsmodels/regression/linear_model.py:1751: RuntimeWarning: divide by zero encountered in true_divide
  return 1 - (np.divide(self.nobs - self.k_constant, self.df_resid)
/home/ismail/anaconda3/lib/python3.7/site-packages/statsmodels/regression/linear_model.py:1752: RuntimeWarning: invalid value encountered in double_scalars
  * (1 - self.rsquared))
/home/ismail/anaconda3/lib/python3.7/site-packages/statsmodels/regression/linear_model.py:1673: RuntimeWarning: divide by zero encountered in double_scalars
  return np.dot(wresid, wresid) / self.df_resid
/home/ismail/anaconda3/lib/python3.7/site-packages/scipy/stats/stats.py:1542: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=13
  "anyway, 

ValueError: resids must contain at least 2 elements